In [1]:
import openai 
import pandas as pd 
import numpy as np
import json
from datetime import datetime
from dfply import *
from plotnine import *

In [2]:
conversations = pd.read_csv("cornell-dialog-data/movie_conversations.tsv", encoding='utf-8-sig', sep="\t", header = None)
titles = pd.read_csv("cornell-dialog-data/movie_titles_metadata.tsv", encoding='utf-8-sig', sep="\t", header = None)
lines_raw = pd.read_csv("cornell-dialog-data/movie_lines.tsv", encoding='utf-8-sig', header = None)
lines_list = lines_raw[0].str.split('\t')
characters_raw = pd.read_csv("cornell-dialog-data/movie_characters_metadata.tsv", encoding='utf-8-sig', header = None)
characters_list = characters_raw[0].str.split('\t')